In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data
import torchvision.models as models
import matplotlib.image as pli
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from PIL import Image

In [ ]:
my_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

file_path = './TinyImageNet'

In [ ]:
class ImageSet(data.Dataset):
    def __init__(self):
        self.length = 100 * 1000

    def __getitem__(self, index):
        # print(index)
        label = int(index / 1000)
        img_count = index % 1000
        # print(label)
        # print(img_count)
        img = Image.open(f'{file_path}/train/{label}/{label}_{img_count}.jpg')
        # img.show()
        img = my_transform(img)
        # print(img.size())
        # exit(0)
        return img, label

    def __len__(self):
        return self.length

In [ ]:
train_loader = data.DataLoader(ImageSet(), batch_size=4, shuffle=True)
test_loader = data.DataLoader(ImageSet(), batch_size=1, shuffle=True)

In [13]:
convNet = models.resnet18()
convNet.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
convNet.maxpool = nn.Sequential()
convNet.fc = nn.Linear(512,100)
print(convNet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Sequential()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),

In [ ]:
# 加载模型， 请谨慎操作， 会覆盖在内存中的模型
convNet = ConvNet()
convNet.load_state_dict(torch.load('./ConvNet.model'))
convNet.eval()

In [11]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convNet.parameters(), lr=0.005)


In [ ]:
for i, (images, labels) in enumerate(train_loader):
    outputs = convNet.forward(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    predict = torch.argmax(nn.functional.softmax(outputs, dim=1), dim=1)
    print(f"i = {i},  loss = {loss},  accuracy = {sum(labels == predict)/labels.size(0)}")

In [ ]:
# 保存模型， 请谨慎操作， 会覆盖文件中的模型
torch.save(convNet.state_dict(), './ConvNet.model')